In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import glob
import plotly.io as pio

# Open Plotly charts in a separate browser window/tab
pio.renderers.default = "browser"

# Load CSV files from dashboard folder
# dashboard_folder = r"C:\Users\phuynh\Projects\robotray\from_hugo\nephe_150"
dashboard_folder = r"C:\Users\phuynh\Projects\robotray\from_hugo\2026_02_09_tray"

dashboard_files = sorted(glob.glob(f"{dashboard_folder}/*.csv"))

print(f"Found {len(dashboard_files)} dashboard CSV files")

# Create subplots: 2 rows x 3 cols
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        "Mining High Voltage Spectra", 
        "Mining Low Voltage Spectra",
        "",
        "Soil High Voltage Spectra",
        "Soil Mid Voltage Spectra",
        "Soil Low Voltage Spectra"
    ),
    x_title="Energy (keV)",
    y_title="Intensity (CPS)",
    specs=[[{}, {}, {}], [{}, {}, {}]]
)

# Helper function to determine app type (Mining or Soil)
def get_app_type(filename):
    if "Mining" in filename:
        return "Mining"
    elif "Soil" in filename:
        return "Soil"
    return None

# Helper function to get the correct intensity column name
def get_intensity_column(df):
    if 'Intensity (CPS)' in df.columns:
        return 'Intensity (CPS)'
    elif 'Intensity (cps)' in df.columns:
        return 'Intensity (cps)'
    else:
        return None

# Helper function to check if file has spectral data
def has_spectral_data(df):
    return 'Energy (keV)' in df.columns and get_intensity_column(df) is not None

Found 894 dashboard CSV files


In [29]:
# Plot dashboard files (darker, more visible)
mining_hv = []
mining_lv = []
soil_hv = []
soil_mid = []
soil_lv = []

for csv_file in dashboard_files:
    filename = Path(csv_file).name
    
    # Skip files containing "chemistry" or "photo"
    if "chemistry" in filename.lower() or "photo" in filename.lower():
        print(f"[SKIPPED] {filename}")
        continue
    
    df = pd.read_csv(csv_file)
    
    # Skip files without spectral data
    if not has_spectral_data(df):
        print(f"[SKIPPED - No spectral data] {filename}")
        continue
    
    intensity_col = get_intensity_column(df)
    test_num = filename.split("_")[0]
    app_type = get_app_type(filename)

    if app_type == "Mining":
        if "HighVoltage" in filename:
            print(f"[Dashboard Mining HV] Adding {filename}")
            mining_hv.append(df[["Energy (keV)", intensity_col]].rename(columns={intensity_col: test_num}))
            fig.add_trace(
                go.Scatter(x=df["Energy (keV)"], y=df[intensity_col],
                          mode='lines', name=f"Dashboard {test_num}", opacity=1.0,
                          line=dict(width=2, color='gray'), 
                          legendgroup='dashboard_mining', showlegend=False),
                row=1, col=1
            )
        elif "LowVoltage" in filename:
            print(f"[Dashboard Mining LV] Adding {filename}")
            mining_lv.append(df[["Energy (keV)", intensity_col]].rename(columns={intensity_col: test_num}))
            fig.add_trace(
                go.Scatter(x=df["Energy (keV)"], y=df[intensity_col],
                          mode='lines', name=f"Dashboard {test_num}", opacity=1.0,
                          line=dict(width=2, color='gray'), 
                          legendgroup='dashboard_mining', showlegend=False),
                row=1, col=2
            )
    elif app_type == "Soil":
        if "HighVoltage" in filename:
            print(f"[Dashboard Soil HV] Adding {filename}")
            soil_hv.append(df[["Energy (keV)", intensity_col]].rename(columns={intensity_col: test_num}))
            fig.add_trace(
                go.Scatter(x=df["Energy (keV)"], y=df[intensity_col],
                          mode='lines', name=f"Dashboard {test_num}", opacity=1.0,
                          line=dict(width=2, color='gray'), 
                          legendgroup='dashboard_soil', showlegend=False),
                row=2, col=1
            )
        elif "MidVoltage" in filename:
            print(f"[Dashboard Soil Mid] Adding {filename}")
            soil_mid.append(df[["Energy (keV)", intensity_col]].rename(columns={intensity_col: test_num}))
            fig.add_trace(
                go.Scatter(x=df["Energy (keV)"], y=df[intensity_col],
                          mode='lines', name=f"Dashboard {test_num}", opacity=1.0,
                          line=dict(width=2, color='gray'), 
                          legendgroup='dashboard_soil', showlegend=False),
                row=2, col=2
            )
        elif "LowVoltage" in filename:
            print(f"[Dashboard Soil LV] Adding {filename}")
            soil_lv.append(df[["Energy (keV)", intensity_col]].rename(columns={intensity_col: test_num}))
            fig.add_trace(
                go.Scatter(x=df["Energy (keV)"], y=df[intensity_col],
                          mode='lines', name=f"Dashboard {test_num}", opacity=1.0,
                          line=dict(width=2, color='gray'), 
                          legendgroup='dashboard_soil', showlegend=False),
                row=2, col=3
            )

def add_median_band(frames, row, col, label):
    if not frames:
        return
    merged = frames[0]
    for frame in frames[1:]:
        merged = merged.merge(frame, on="Energy (keV)", how="inner")

    energy = merged["Energy (keV)"]
    values = merged.drop(columns=["Energy (keV)"])
    median = values.median(axis=1)
    q25 = values.quantile(0.25, axis=1)
    q75 = values.quantile(0.75, axis=1)

    fig.add_trace(
        go.Scatter(
            x=energy,
            y=q75,
            mode="lines",
            line=dict(width=0),
            showlegend=False,
            hoverinfo="skip"
        ),
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter(
            x=energy,
            y=q25,
            mode="lines",
            fill="tonexty",
            fillcolor="rgba(80,80,80,0.35)",
            line=dict(width=0),
            name=f"{label} 25–75%",
            legendgroup=label
        ),
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter(
            x=energy,
            y=median,
            mode="lines",
            line=dict(width=3, color="rgb(0,0,0)"),
            name=f"{label} median",
            legendgroup=label
        ),
        row=row, col=col
    )

add_median_band(mining_hv, 1, 1, "Mining HV")
add_median_band(mining_lv, 1, 2, "Mining LV")
add_median_band(soil_hv, 2, 1, "Soil HV")
add_median_band(soil_mid, 2, 2, "Soil Mid")
add_median_band(soil_lv, 2, 3, "Soil LV")

# Update axis labels
for row in [1, 2]:
    for col in [1, 2, 3]:
        fig.update_xaxes(title_text="Energy (keV)", row=row, col=col)
        fig.update_yaxes(title_text="Intensity (CPS)", row=row, col=col)

fig.update_layout(height=900, width=1600)
fig.show()

[Dashboard Mining HV] Adding 003004_2026_02_10_13225221_MiningHighVoltage_CPS.csv
[Dashboard Mining LV] Adding 003004_2026_02_10_13225221_MiningLowVoltage_CPS.csv
[Dashboard Soil HV] Adding 003004_2026_02_10_13225757_SoilHighVoltage_CPS.csv
[Dashboard Soil LV] Adding 003004_2026_02_10_13225757_SoilLowVoltage_CPS.csv
[Dashboard Soil Mid] Adding 003004_2026_02_10_13225757_SoilMidVoltage_CPS.csv
[SKIPPED] 003004_2026_02_10_13225763_chemistry.csv
[Dashboard Mining HV] Adding 003005_2026_02_10_13230243_MiningHighVoltage_CPS.csv
[Dashboard Mining LV] Adding 003005_2026_02_10_13230243_MiningLowVoltage_CPS.csv
[Dashboard Soil HV] Adding 003005_2026_02_10_13230758_SoilHighVoltage_CPS.csv
[Dashboard Soil LV] Adding 003005_2026_02_10_13230758_SoilLowVoltage_CPS.csv
[Dashboard Soil Mid] Adding 003005_2026_02_10_13230758_SoilMidVoltage_CPS.csv
[SKIPPED] 003005_2026_02_10_13230764_chemistry.csv
[Dashboard Mining HV] Adding 003006_2026_02_10_13231242_MiningHighVoltage_CPS.csv
[Dashboard Mining LV] Ad